In [1]:
#from yt_dlp import YoutubeDL
import os
#import re

In [5]:
from pytube import YouTube
from moviepy import VideoFileClip

### Audio Conversion from Video

In [ ]:
video_path = "/home/pms/llm_project/mm_rag_esg_financial_project/downloads_vdo/Invest_with_Conscience_ ESG_Investing.mp4"
audio_path = os.path.splitext(video_path)[0] + '.mp3'

video_clip = VideoFileClip(video_path)
video_clip.audio.write_audiofile(audio_path,codec='mp3')
video_clip.close()


### Transcriptions Generation with Whisper

In [ ]:
!pip -qqq install openai-whisper

In [6]:
import whisper
import torch

In [3]:
device = "cude" if torch.cuda.is_available() else "cpu"
whisper_model = whisper.load_model("medium",device=device)

/home/pms/llm_project/mm_rag_esg_financial_project/esg/lib/python3.10/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch

In [4]:
import glob
# Initialize the AudioTranscriber
#transcriber = AudioTranscriber(input_folder=r"./data")

# Initialize your Whisper model
#transcriber.whisper_model = whisper_model

# Transcribe all audios in the input folder
#for file in glob.glob(f'{os.getcwd()}/data/*.mp3'):
transcriptions_dict = whisper_model.transcribe("/home/pms/llm_project/mm_rag_esg_financial_project/data/ESG_investing_complete.mp3")

#print(transcriptions_dict)
#transcriptions_dict

# for _, data in transcriptions_dict.items():
#     #print(f"URL: {url}")
#     print(f"Audio file: {data['audio_path']}")
#     print(f"Transcription: {data['transcription'][:100]}...")  # Print first 100 characters
#     print("---")

/home/pms/llm_project/mm_rag_esg_financial_project/esg/lib/python3.10/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [6]:
transcriptions_dict['text']

" ESG, real or marketing? It's a complete fraud. Complete fraud. It's so ridiculous. Governance has been addressed. That's useful. But this idea that you're going to get a stamp that says, oh, listen, my supplier, I've offset their carbon credits and now I understand my... It's a joke. It's jargon. And I think what people are doing right now is using it as a way to, for example, if you can paint yourself as ESG, in Europe you can essentially borrow money from the ECB at negative rates. I can do you a massage now. It's a carry-trade. He doesn't want you touching him. Coronavirus. Go Chamath. Go Chamath. Hold on. But I personally believe in climate change. I know. I think we need to do something. And so the problem with ESG is it's going to take years for this... But when you hear JP Morgan yesterday say they're not going to finance fossil fuels or you hear Ed Bastien at Delta say he's going to spend $100 million, real money by the way, effectively buying carbon offsets and investing in 

### PDF Document Parsing

In [42]:
from unstructured.partition.pdf import partition_pdf

In [6]:
esg_report_path = "/home/pms/llm_project/mm_rag_esg_financial_project/pdf/Bidding_Document_UWS_AGRC.pdf"

In [43]:
import pickle

raw_pdf_elements=partition_pdf(
    filename=esg_report_path,
    strategy="hi_res",
    extract_images_in_pdf=True,
    extract_image_block_types=["Image", "Table"],
    extract_image_block_to_payload=False,
    extract_image_block_output_dir="extracted_data",
)

# Save the extracted data to a file
with open("extracted_pdf_data.pkl", "wb") as file:
    pickle.dump(raw_pdf_elements, file)

In [44]:
import pickle
with open("extracted_pdf_data.pkl", "rb") as file:
    loaded_pdf_elements = pickle.load(file)


In [ ]:
loaded_pdf_elements

In [ ]:
Header=[]
Footer=[]
Title=[]
NarrativeText=[]
Text=[]
ListItem=[]
img=[]
tab=[]
for element in raw_pdf_elements:
  if "unstructured.documents.elements.Header" in str(type(element)):
            Header.append(str(element))
  elif "unstructured.documents.elements.Footer" in str(type(element)):
            Footer.append(str(element))
  elif "unstructured.documents.elements.Title" in str(type(element)):
            Title.append(str(element))
  elif "unstructured.documents.elements.NarrativeText" in str(type(element)):
            NarrativeText.append(str(element))
  elif "unstructured.documents.elements.Text" in str(type(element)):
            Text.append(str(element))
  elif "unstructured.documents.elements.ListItem" in str(type(element)):
            ListItem.append(str(element))
  elif "unstructured.documents.elements.Image" in str(type(element)):
            img.append(str(element))
  elif "unstructured.documents.elements.Table" in str(type(element)):
            tab.append(str(element))

### Extracting Textual Component

In [46]:
from unstructured.documents.elements import NarrativeText

In [47]:
def extract_text_with_metadata(raw_data, source_document):

    text_data = []
    paragraph_counters = {}

    for element in raw_data:
        if isinstance(element, NarrativeText):
            page_number = element.metadata.page_number

            if page_number not in paragraph_counters:
                paragraph_counters[page_number] = 1
            else:
                paragraph_counters[page_number] += 1

            paragraph_number = paragraph_counters[page_number]

            text_content = element.text
            text_data.append({
                "source_document": source_document,
                "page_number": page_number,
                "paragraph_number": paragraph_number,
                "text": text_content
            })

    return text_data

In [48]:
extracted_text = extract_text_with_metadata(raw_pdf_elements, esg_report_path)

In [ ]:
extracted_text

In [102]:
prefixed_extracted_text = ["summarize: " + text_dict["source_document"] for text_dict in extracted_text]

### Extracting Image Components

In [50]:
from unstructured.documents.elements import Image

In [51]:
def extract_image_metadata(raw_data, source_document):
    image_data = []

    for element in raw_data:
        if isinstance(element, Image):
            page_number = element.metadata.page_number
            image_path = element.metadata.image_path if hasattr(element.metadata, 'image_path') else None

            image_data.append({
                "source_document": source_document,
                "page_number": page_number,
                "image_path": image_path
            })

    return image_data

In [52]:
extracted_image_data = extract_image_metadata(raw_pdf_elements, esg_report_path)

In [53]:
import matplotlib.pyplot as plt
from PIL import Image
import math

In [54]:
def display_images_from_metadata(extracted_image_data, images_per_row=4):
    valid_images = [img for img in extracted_image_data if img['image_path']]
    if not valid_images:
        print("No valid image data available.")
        return

    num_images = len(valid_images)
    num_rows = math.ceil(num_images / images_per_row)

    fig, axes = plt.subplots(num_rows, images_per_row, figsize=(20, 5*num_rows))
    axes = axes.flatten() if num_rows > 1 else [axes]

    for ax, img_data in zip(axes, valid_images):
        try:
            img = Image.open(img_data['image_path'])
            ax.imshow(img)
            ax.axis('off')
            ax.set_title(f"Page {img_data['page_number']}", fontsize=10)
        except Exception as e:
            print(f"Error loading image {img_data['image_path']}: {str(e)}")
            ax.text(0.5, 0.5, f"Error loading image\n{str(e)}", ha='center', va='center')
            ax.axis('off')

    for ax in axes[num_images:]:
        fig.delaxes(ax)

    plt.tight_layout()
    plt.show()

In [ ]:
display_images_from_metadata(extracted_image_data)

### Extracting Table Components

In [56]:
from unstructured.documents.elements import Table

In [57]:
def extract_table_metadata(raw_data, source_document):
    table_data = []

    for element in raw_data:
        if isinstance(element, Table):
            page_number = element.metadata.page_number

            # Extract table content as a string
            table_content = str(element)

            table_data.append({
                "source_document": source_document,
                "page_number": page_number,
                "table_content": table_content
            })

    return table_data

In [58]:
extracted_table_data = extract_table_metadata(loaded_pdf_elements, esg_report_path)

In [ ]:
extracted_table_data

In [ ]:
# import os
# import json
# config_data = json.load(open("./config.json"))
# #MISTRALAI_API_KEY = config_data["MISTRAL_API_KEY"]
# GROQ_API_KEY = config_data["GROQ_API_KEY"]
# #os.environ["MISTRALAI_API_KEY"] = MISTRALAI_API_KEY
# os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [110]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,BartForCausalLM,AutoModel

model_name = "google-t5/t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir="/home/pms/llm_project/mm_rag_esg_financial_project/saved_model")
model_bart = AutoModelForSeq2SeqLM.from_pretrained(model_name,cache_dir="/home/pms/llm_project/mm_rag_esg_financial_project/saved_model")

In [61]:
from transformers import AutoTokenizer, AutoModel

model_name = "microsoft/layoutlmv3-base"
tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir="/home/pms/llm_project/mm_rag_esg_financial_project/saved_model")
model_lvm = AutoModel.from_pretrained(model_name,cache_dir="/home/pms/llm_project/mm_rag_esg_financial_project/saved_model")

In [ ]:
model_bart

In [113]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# Create a Hugging Face pipeline for the LLaVA model
pipe = pipeline(
    "summarization",
    model=model_bart,
    tokenizer=tokenizer,
    max_length=100,
    truncation=True
    #device=0  # Set this to -1 if you don't have a GPU
)

# Integrate the Hugging Face pipeline with Langchain
llm = HuggingFacePipeline(pipeline=pipe)


Device set to use cpu


In [114]:
llm

HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.SummarizationPipeline object at 0x7f07bf5b5e10>)

### Summary Generation

In [93]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

### Text Summary

In [75]:
# Prompt
prompt_text = """You are an assistant tasked with summarizing text for retrieval. \
    These summaries will be embedded and used to retrieve the raw text elements. \
    Give a concise summary of the text that is well optimized for retrieval.text: {element} """

In [68]:
prompt_text

'You are an assistant tasked with summarizing text for retrieval.     These summaries will be embedded and used to retrieve the raw text elements.     Give a concise summary of the text that is well optimized for retrieval.text: {element} '

In [76]:
prompt = ChatPromptTemplate.from_template(prompt_text)

In [23]:
prompt

ChatPromptTemplate(input_variables=['element'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['element'], input_types={}, partial_variables={}, template='You are an assistant tasked with summarizing text for retrieval.     These summaries will be embedded and used to retrieve the raw text elements.     Give a concise summary of the text that is well optimized for retrieval.text: {element} '), additional_kwargs={})])

In [115]:
summarize_chain = {"element": lambda x: x} | prompt | llm | StrOutputParser()

In [ ]:
text_summaries = []
text_summaries = summarize_chain.batch(extracted_text,{"max_concurrency":5})

In [81]:
text_summaries

[]

In [26]:
clean_text_summaries = [sub.replace("""Human: You are an assistant tasked with summarizing text for retrieval. \
    These summaries will be embedded and used to retrieve the raw text elements. \
    Give a concise summary of the text that is well optimized for retrieval.text:""", "") for sub in text_summaries]

In [30]:
clean_text_summaries = [sub.replace(f"\"\s", "") for sub in clean_text_summaries]

In [ ]:
clean_text_summaries

### Table Data Summarization

In [117]:
tables_summarizer_prompt = """
Summarize the key information from the following table extracted from a tender document. Focus on the most important details such as bidder requirements, project deadlines, cost estimates, evaluation criteria, and any other significant values that are essential for understanding the tender's terms.

Table:{table_content}

Provide a concise summary that highlights the critical points necessary for a quick review of the tender's specifications.

"""

In [ ]:
summarize_chain = {"element": lambda x: x} | prompt | model_lvm | StrOutputParser()

In [120]:
table_summaries = []
table_summaries = summarize_chain.batch(extracted_table_data,{"max_concurrency":5})

KeyboardInterrupt: 